Install  necessary libraries

In [1]:
# !pip install nltk
# !pip install json
# !pip install gensim
# nltk.download('punkt')

Import necessary libraries

In [2]:
import json
import nltk
import gensim
import numpy as np
from gensim import corpora, models, similarities
import pickle

Load word2vec model from gensim

In [3]:
model = gensim.models.Word2Vec.load('word2vec.bin');

C:\Users\ANDITYAARIFIANTO\Anaconda3\envs\tensorflow-gpu\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


Load Conversation dataset

In [4]:
file=open('conversation3.json');
data = json.load(file)
cor=data["conversations"];

Create a series of couple corpus as input data and target. <br>
One corpus of conversation is the target for the previous conversation

In [5]:
x=[]
y=[]

for i in range(len(cor)):
    for j in range(len(cor[i])):
        if j<len(cor[i])-1:
            x.append(cor[i][j]);
            y.append(cor[i][j+1]);

Example of a conversation and it's target

In [6]:
idx = 21
print(x[idx],':', y[idx])

I'd like to buy a fried chicken : Sure


Tokenize all input and target<br>
create token for *'end-of-sentence'*

In [7]:
tok_x=[]
tok_y=[]
for i in range(len(x)):
    tok_x.append(nltk.word_tokenize(x[i].lower()))
    tok_y.append(nltk.word_tokenize(y[i].lower()))

sentence_end=np.ones((300,),dtype=np.float32) 

Vectorize all sentece token using gensim word2vec model

In [8]:
vec_x=[]
for sent in tok_x:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_x.append(sentvec)
    
vec_y=[]
for sent in tok_y:
    sentvec = [model[w] for w in sent if w in model.wv.vocab]
    vec_y.append(sentvec)           
    

C:\Users\ANDITYAARIFIANTO\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\ANDITYAARIFIANTO\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Example of a conversation pair and it's token lengths

In [9]:
print(x[idx],':', y[idx])
print(len(vec_x[idx]),':', len(vec_y[idx]))

I'd like to buy a fried chicken : Sure
8 : 1


Limit the maximum length of a sentence vector is 10

In [10]:
for tok_sent in vec_x:
    tok_sent[9:]=[]
    tok_sent.append(sentence_end)
    

for tok_sent in vec_x:
    if len(tok_sent)<10:
        for i in range(10-len(tok_sent)):
            tok_sent.append(sentence_end)    

In [11]:
for tok_sent in vec_y:
    tok_sent[9:]=[]
    tok_sent.append(sentence_end)
    

for tok_sent in vec_y:
    if len(tok_sent)<10:
        for i in range(10-len(tok_sent)):
            tok_sent.append(sentence_end)             

Example of a conversation pair and it's token lengths after limit it to 10 tokens

In [12]:
print(x[idx],':', y[idx])
print(len(vec_x[idx]),':', len(vec_y[idx]))

I'd like to buy a fried chicken : Sure
10 : 10


Write the vectors to an output file

In [13]:
with open('conversation.pickle','wb') as f:
    pickle.dump([vec_x,vec_y],f) 